In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, TabularDataset, BucketIterator

from dataset.mtgcards import RuleText
from utils.preprocess import fields_for_rule_text

import random
import math
import time
import os

d:\Miniconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SRC, TRG = fields_for_rule_text()
fields = {'src': ('src', SRC), 'trg': ('trg', TRG)}

train_data, valid_data, test_data = RuleText.splits(fields=fields, version='v2.2')

In [3]:
SRC.build_vocab(train_data, min_freq = 4)
TRG.build_vocab(train_data, min_freq = 4)
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (zh) vocabulary: {len(TRG.vocab)}")

for x in random.sample(list(train_data), 3):
    print(x.src, x.trg)

Unique tokens in source (en) vocabulary: 1294
Unique tokens in target (zh) vocabulary: 1949
['flash'] ['flash']
['<', '7', '>'] ['<', '7', '>']
['whenever', 'another', 'creature', 'dies', ',', 'put', 'two', '+', '1', '/', '+', '1', 'counters', 'on', '<', '4', '>', '.'] ['每当', '另', '一个', '生物', '死去', '时', '，', '在', '<', '4', '>', '上', '放置', '两', '个', '+', '1', '/', '+', '1', '指示物', '。']


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device)

tmp = next(iter(train_iterator))
print(tmp)

cpu

[torchtext.legacy.data.batch.Batch of size 128]
	[.src]:('[torch.LongTensor of size 29x128]', '[torch.LongTensor of size 128]')
	[.trg]:[torch.LongTensor of size 40x128]


In [5]:
from models.model4.definition import Encoder, Attention, Decoder, Seq2Seq
from models.model4.train import init_weights, train, evaluate
from utils import count_parameters, train_loop

In [6]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

model.apply(init_weights)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,758,045 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

train_loop(model, optimizer, criterion, train, evaluate,
           train_iterator, valid_iterator, 
           save_path='result/', file_name='model4-rule-v2.2.pt', load_before_train=True)

In [8]:
from utils.translate import Translator
from models.model4.definition import beam_search
model.load_state_dict(torch.load('result/model4-rule-v2.2.pt', map_location=torch.device(device)))
T = Translator(SRC, TRG, model, device, beam_search)
torch.save(T,'result/model4-T-v2.2.pt')

In [9]:
data = 'Whenever <1> becomes attached to a creature, for as long as <1> remains attached to it, you may have that creature become a copy of another target creature you control.'
data = 'target creature gets - 1 / - 1 until end of turn .'
ret, prob = T.translate(data, max_len=100)
print(*ret[:3], sep='\n')

['目标', '生物', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '-', '1', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '-', '1', '1', '-', '1', '直到', '回合', '结束', '。', '<eos>']


In [10]:
from utils import show_samples
long_data = [x for x in test_data.examples if len(x.src) > 20]
print(f'Number of samples: {len(long_data)}')
show_samples(long_data, T, n=3, beam_size=3)

Number of samples: 355
src: [at the beginning of your next upkeep , you may cast this card from exile without paying its mana cost . ] trg = [在你的下一个维持开始时，你可以从放逐区施放此牌，且不须支付其法术力费用。]
在你的下一个维持开始时，你可以从放逐区施放此牌，且不需支付其法术力费用。<eos> 	[probability: 0.63112]
在你的下一个维持开始时，你可以从放逐区施放此牌，且不须支付其法术力费用。<eos> 	[probability: 0.14685]
在你下一个维持开始时，你可以从放逐区施放此牌，且不需支付其法术力费用。<eos> 	[probability: 0.03197]

src: [whenever a creature enters the battlefield under an opponent 's control , you may attach < 6 > to that creature . ] trg = [每当一个生物在对手的操控下进场时，你可以将<6>结附在该生物上。]
每当一个生物在对手的操控下进战场时，你可以将<6>装备在该生物上。<eos> 	[probability: 0.06546]
每当一个生物在对手的操控下进战场时，你可以将<6>在该生物上。<eos> 	[probability: 0.06375]
每当一个生物在对手的操控下进战场时，你可以将<6>对该生物上。<eos> 	[probability: 0.05898]

src: [non - horror creatures with slime counters on them lose all abilities and have base power and toughness 2 / 2 . ] trg = [所有其上有黏菌指示物的非惊惧兽生物都失去所有异能，且基础力量与防御力均为2/2。]
由其上有且其上有指示物的生物均失去所有异能与防御力为2/2。<eos> 	[probability: 0.00012]
由其上有且其上有指示物的生物均失去所有基础力量与防御力为2/2。<eos> 	[prob

In [11]:
from dataset.mtgcards import TestSets
from utils import calculate_bleu
from torchtext.legacy.data import Field
from models.card_name_detector.definition import TrainedDetector
from utils.translate import sentencize, CardTranslator

fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
test_data = TestSets.load(fields)

D = TrainedDetector()

path: d:\Desktop\mtg-cards-translation\models\card_name_detector


In [13]:
def sentencize(text: str):
    ignore = {' ', '(', ')', '\n'}
    while len(text) and text[0] in ignore:
        text = text[1:]
    if len(text) == 0:
        return []
    
    r = 0
    delims = {'.', '\n', '('}
    ignore = False
    while r < len(text):
        if text[r] == '\"':
            ignore = not ignore
        if not ignore and text[r] in delims:
            break
        r += 1
    
    if r < len(text) and text[r] == '.':
        return [text[:r + 1]] + sentencize(text[r + 1:])
    return [text[:r]] + sentencize(text[r:])
def preprocess(x:str):
    x = D.annotate(x).removeprefix(' ')
    print(f'[after preprocess]:{x}')
    return x
def postprocess(x:str):
    return x.replace('<', '').replace('>', '')

import re
class CTHelper:
    def __init__(self, name_detector, dictionary={}) -> None:
        self.D = name_detector
        self.dictionary = dictionary
    
    def preprocess(self, x:str):
        self.tag2str = {}
        x = D.annotate(x).removeprefix(' ') # x become lowercase after go through detector
        m = re.search('<[^0-9>]+>', x)
        id = 0
        while m:
            l, r = m.span()
            tag = '<' + str(id) + '>'
            self.tag2str[tag] = x[l:r]
            x = x[:l] + tag + x[r:]
            id += 1
            m = re.search('<[^0-9>]+>', x)

        for s in self.dictionary.keys():
            m = re.search(s, x)
            if m:
                tag = '<' + str(id) + '>'
                self.tag2str[tag] = s
                x = x.replace(s, tag)
                id += 1

        #print(f'[  after preprocess]:{x}')
        return x

    def postprocess(self, x:str):
        #print(f'[before postprocess]:{x}')
        for tag, s in self.tag2str.items():
            x = x.replace(tag, self.dictionary[s] if s in self.dictionary else s)
        return x

dic = {}
dic = {'oil':'烁油', 'rebel':'反抗军','compleated':'完化'}
helper = CTHelper(D, dic)
CT = CardTranslator(sentencize, T, preprocess=lambda x: helper.preprocess(x), postprocess=lambda x:helper.postprocess(x))

example = random.sample(list(test_data), 1)[0]
example = list(test_data)[237]
print(vars(example))
ret=CT.translate(' '.join(example.src))
print(ret+'\n')
for example in random.sample(list(test_data), 3):
    print(vars(example))
    ret = CT.translate(' '.join(example.src))
    print(ret + '\n')

{'src': ['Gain', 'control', 'of', 'target', 'creature', 'with', 'mana', 'value', 'X', 'or', 'less.', 'If', 'X', 'is', '5', 'or', 'more,', 'create', 'a', 'token', "that's", 'a', 'copy', 'of', 'that', 'creature.'], 'trg': ['获得目标法术力值等于或小于X的生物之操控权。如果X等于或大于5，则派出一个衍生物，此衍生物为该生物的复制品。']}
获得目标总法术力费用等于或小于x的生物之操控权。 如果x等于或大于5，则将一个衍生物放进战场，此衍生物为该生物的复制品。

{'src': ['For', 'Mirrodin!', '(When', 'this', 'Equipment', 'enters', 'the', 'battlefield,', 'create', 'a', '2/2', 'red', 'Rebel', 'creature', 'token,', 'then', 'attach', 'this', 'to', 'it.)\nEquipped', 'creature', 'gets', '+2/+1', 'and', 'has', 'vigilance.\nEquip', '{3}{W}', '({3}{W}:', 'Attach', 'to', 'target', 'creature', 'you', 'control.', 'Equip', 'only', 'as', 'a', 'sorcery.)'], 'trg': ['秘罗万岁！（当此武具进战场时，派出一个2/2红色反抗军衍生生物，然后将它贴附于其上。）', '佩带此武具的生物得+2/+1且具有警戒异能。', '佩带{3}{W}（{3}{W}：贴附在目标由你操控的生物上。只能于法术时机佩带。）']}
<unk><unk><unk><unk>融合。 当此武具进战场时，将一个2/2红色反抗军衍生生物放进战场，然后将它装备上去。 佩带此武具的生物得+2/+1且具有警戒异能。 佩带{3}{w} {3}{w}：装备在目标由你操控的生物上。 佩带的时机视同法术。

{'src': ['Tramp

In [14]:
from utils import calculate_testset_bleu
calculate_testset_bleu(list(test_data)[:100], CT)

100%|██████████| 100/100 [00:39<00:00,  2.53it/s]


0.6877907101188122